In [14]:
import os
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import scipy.stats
import seaborn as sns
import statsmodels.api as sm
from dateutil.relativedelta import relativedelta
from plotly.subplots import make_subplots
from scipy.stats import normaltest
from sklearn import preprocessing

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)
pd.options.mode.chained_assignment = None
# seaborn.set(rc={'figure.figsize': (30, 10)})
plt.ion()

# SQLite database setup


In [15]:
import import_ipynb
import setup_sqlite
# %run setup_sqlite.ipynb

In [16]:
setup_sqlite.drop_tables()
setup_sqlite.create_tables()

# Data Acquisition

Some of the data was retrieved manually from the Transitapp website (usually the earlier dates in this project's record).

Most of the data was retrieved from https://transitapp.com/rats.csv at 6:00PM EST via a cronjob. This may mean that the dataset name might be mismatched from what the website actually gives you (typically off by one day).

However all of the datasets are non-overlapping.


## Reformatting raw data files from cloud server storage


In [17]:
root_data_dir = "rat_data"

data_files = os.listdir(root_data_dir)
formatted_files = []

# Remove helper sh files
for file in data_files:
    if file.endswith(".sh"):
        data_files.remove(file)

# Standardize name of files and remove conflicting ones
for i, file in enumerate(data_files):
    if file.startswith("rats"):
        date = file.split(" ")[1].split(".")[0]
        dt_end = datetime.strptime(date, "%Y-%m-%d")
        dt_start = dt_end - relativedelta(days=30)

        new_name = f"Transit app rat reports - {dt_start.strftime('%Y-%m-%d')} to {dt_end.strftime('%Y-%m-%d')}.csv"
        old_path = os.path.join("rat_data", file)
        new_path = os.path.join("rat_data", new_name)

        if new_name in data_files:
            print(f"CONFLICT -- {file}, {data_files[data_files.index(new_name)]}")
        else:
            print(f"{old_path} --> {new_path}")
            os.rename(old_path, new_path)
            formatted_files.append(new_name)
    else:
        formatted_files.append(file)

formatted_files = sorted(formatted_files)

CONFLICT -- rats 2023-11-11.csv, Transit app rat reports - 2023-10-12 to 2023-11-11.csv
CONFLICT -- rats 2023-11-12.csv, Transit app rat reports - 2023-10-13 to 2023-11-12.csv


## Collation of datasets and adding some addition columns of information


In [18]:
for file in formatted_files:
    if file.endswith(".csv"):
        full_path = os.path.join(root_data_dir, file)
        tokens = full_path.split(" ")

        date_start = tokens[-3]
        date_end = tokens[-1].split(".")[0]

        rat_sv = pd.read_csv(full_path)
        rat_sv["station_name"] = rat_sv["station_name"].replace(r'\s+', ' ', regex=True)
        print(rat_sv)
        # rat_sv = rat_sv.drop(["lines_served"], axis=1)

        stations_rows = []
        sightings_rows = []

        for index, row in rat_sv.iterrows():
            stations_rows.append(
                [row["station_id"], row["station_name"], row["lines_served"], None, None]
            )
            sightings_rows.append(
                [
                    row["station_id"],
                    row["so_many"],
                    row["one_or_two"],
                    row["none"],
                    date_start,
                    date_end,
                ]
            )

        setup_sqlite.insert_stations(stations_rows)
        setup_sqlite.insert_sightings(sightings_rows)

     station_id          station_name  so_many  one_or_two  none lines_served
0         17496                191 St        3           7     2            1
1         17700              Grant Av        5           8     3            A
2         17497                181 St        9          21    10            1
3         17872                Bowery        8           6     5          J Z
4         20055   Kingston-Throop Avs       16          10    11          A C
..          ...                   ...      ...         ...   ...          ...
375       17820           Beach 25 St        0           0    11            A
376       17864             Seneca Av        0           0    11            M
377       17919              Annadale        0           0    10          SIR
378       20085  Astoria-Ditmars Blvd        0           0    20          N W
379       20161      75 St-Elderts Ln        0           0    12          J Z

[380 rows x 6 columns]
     station_id          station_name  s

In [19]:
mta_all_stations = pd.read_csv("MTA_Subway_Stations_20240304.csv")
mta_all_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GTFS Stop ID           496 non-null    object 
 1   Station ID             496 non-null    int64  
 2   Complex ID             496 non-null    int64  
 3   Division               496 non-null    object 
 4   Line                   496 non-null    object 
 5   Stop Name              496 non-null    object 
 6   Borough                496 non-null    object 
 7   Daytime Routes         496 non-null    object 
 8   Structure              496 non-null    object 
 9   GTFS Latitude          496 non-null    float64
 10  GTFS Longitude         496 non-null    float64
 11  North Direction Label  496 non-null    object 
 12  South Direction Label  496 non-null    object 
 13  ADA                    496 non-null    int64  
 14  ADA Northbound         496 non-null    int64  
 15  ADA So

In [20]:
mta_all_stations[(mta_all_stations["Stop Name"].str.contains("5 Av"))]

,GTFS Stop ID,Station ID,Complex ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Northbound,ADA Southbound,ADA Notes,Georeference
7,R13,8,8,BMT,Astoria,5 Av/59 St,M,N R W,Subway,40.764811,-73.973347,Queens,Downtown,0,0,0,NaN,POINT (-73.973347 40.764811)
68,B22,69,69,BMT,West End,25 Av,Bk,D,Elevated,40.597704,-73.986829,Manhattan,Coney Island,0,0,0,NaN,POINT (-73.986829 40.597704)
260,F07,260,260,IND,Queens Blvd,75 Av,Q,E F,Subway,40.718331,-73.837324,Jamaica,Manhattan,0,0,0,NaN,POINT (-73.837324 40.718331)
276,F12,276,276,IND,Queens Blvd,5 Av/53 St,M,E M,Subway,40.760167,-73.975224,Queens,Downtown,0,0,0,NaN,POINT (-73.975224 40.760167)
467,724,466,609,IRT,Flushing,5 Av,M,7,Subway,40.753821,-73.981963,Queens,Hudson Yards,0,0,0,NaN,POINT (-73.981963 40.753821)


In [21]:
cur = setup_sqlite.get_cursor()

station_latlongs = []
not_found_cnt = 0

for station_id, station_name, station_lines, _, _ in cur.execute("SELECT * from stations"):
    # station_tokens = station_name.split(" ")

    # if station_tokens[0].isnumeric():
    #     # Modify station name to have the numeric suffixes if applicable
    #     if station_tokens[0][-1] == "1":
    #         station_tokens[0] += "st"
    #     elif station_tokens[0][-1] == "2":
    #         station_tokens[0] += "nd"
    #     elif station_tokens[0][-1] == "3":
    #         station_tokens[0] += "rd"
    #     else:
    #         station_tokens[0] += "th"

    # # Translate certain stations
    # if "Washington Hts" in station_name:
    #     station_tokens = ["168th St - Washington Heights"]
    # elif "Cathedral Pkwy" in station_name:
    #     station_tokens = ["Cathedral Parkway-110th St"]
    # elif "Columbus Circle" in station_name:
    #     station_tokens = ["59th St-Columbus Circle"]
    # elif "Times Sq - 42 St / Port Authority Bus Terminal" in station_name:
    #     station_tokens = ["Times Square-42nd St"]
    # elif "14 St / 6 Av" in station_name:
    #     station_tokens = ["14th St"]
    # elif "WTC Cortlandt" in station_name:
    #     station_tokens = ["World Trade Center"]
    #     station_lines = "A C E 2 3"
    # elif "South Ferry / Whitehall" in station_name:
    #     station_tokens = ["South Ferry"]
    #     station_lines = "R 1"
    # elif "Nereid Av" in station_name:
    #     station_tokens = ["238th St-Nereid Av"]
    # elif "Pelham Pkwy" in station_name:
    #     station_tokens = ["Pelham Parkway"]
    # elif "E 180 St" in station_name:
    #     station_tokens = ["East 180th St"]
    # elif "149 St - Grand Concourse" in station_name:
    #     station_tokens = ["149th St-Grand Concourse"]
    # elif "Central Park North (110 St)" in station_name:
    #     station_tokens = ["110th St-Central Park North"]
    # elif "Chambers St / WTC / Park Place / Cortlandt St" in station_name:
    #     station_tokens = ["Park Place"]
    #     station_lines = "A C E 1 2 3"
    # elif "Borough Hall / Court St" in station_name:
    #     station_tokens = ["Court St"]
    #     station_lines = "R 2 3 4 5"
    # elif "Franklin Av / Botanic Garden" in station_name:
    #     station_tokens = ["Botanic Gardens"]
    #     station_lines = "2 3 4 5 FS"
    # elif "Mosholu Pkwy" in station_name:
    #     station_tokens = ["Mosholu Parkway"]
    # elif "161 St - Yankee Stadium" in station_name:
    #     station_tokens = ["Yankee Stadium-161st St"]
    # elif "Pelham Pkwy" in station_name:
    #     station_tokens = ["Pelham Parkway"]
    # elif "E 149 St" in station_name:
    #     station_tokens = ["East 149th St"]
    # elif "Lexington Av / 59 St" in station_name:
    #     station_tokens = ["59th St"]
    # elif "Lexington Av / 51 St" in station_name:
    #     station_tokens = ["51st St"]
    # elif "Grand Central - 42 St" in station_name:
    #     station_tokens = ["Grand Central-42nd St"]
    #     station_lines += " GS"
    # elif "14 St - Union Sq" in station_name:
    #     station_tokens = ["14th St-Union Square"]
    # elif "Astor Pl" in station_name:
    #     station_tokens = ["Astor Place"]
    # elif "Broadway-Lafayette St / Bleecker St" in station_name:
    #     station_tokens = ["Bleecker St"]
    #     station_lines = "B D F M 6"
    # elif "Brooklyn Bridge-City Hall / Chambers St" in station_name:
    #     station_tokens = ["Chambers St"]
    #     station_lines = "J Z 4 5 6"
    # elif "Jackson Hts-Roosevelt Av / 74 St" in station_name:
    #     station_tokens = ["Jackson Heights-Roosevelt Ave"]
    #     station_lines = "E F M R 7"
    # elif "Court Sq - 23 St" in station_name:
    #     station_tokens = ["23rd St-Ely Av"]
    #     station_lines = "E G M 7"
    # elif "42 St - Bryant Pk / 5 Av" in station_name:
    #     station_tokens = ["5th Av"]
    #     station_lines = "B D F M 7"
    # elif "W 4 St - Wash Sq" in station_name:
    #     station_tokens = ["West 4th St"]
    # elif "Jay St - MetroTech" in station_name:
    #     station_tokens = ["Jay St - Borough Hall"]
    #     station_lines = "A C F R"
    # elif "Fort Hamilton Pkwy" in station_name:
    #     station_tokens = ["Fort Hamilton Parkway"]
    # elif "62 St / New Utrecht Av" in station_name:
    #     station_tokens = ["New Utrecht Av"]
    # elif "Bay Pkwy" in station_name:
    #     if set(station_lines) == set("D"):
    #         station_tokens = ["Bay Parkway"]
    #     elif set(station_lines) == set("F"):
    #         station_tokens = ["Bay Parkway-22nd Av"]
    #     elif set(station_lines) == set("D N Q W"):
    #         station_tokens = ["Bay Parkway-22nd Av"]
    #         station_lines = "N"
    # elif "182-183 Sts" in station_name:
    #     station_tokens = ["182nd-183rd Sts"]
    # elif "174-175 Sts" in station_name:
    #     station_tokens = ["174-175th Sts"]
    # elif "34 St - Herald Sq" in station_name:
    #     station_tokens = ["34th St"]
    # elif "Newkirk Plaza" in station_name:
    #     station_tokens = ["Newkirk Av"]
    # elif "Ocean Pkwy" in station_name:
    #     station_tokens = ["Ocean Parkway"]
    # elif "Delancey St / Essex St" in station_name:
    #     station_tokens = ["Delancey St"]
    # elif "4 Av - 9 St" in station_name:
    #     station_tokens = ["4th Av"]
    # elif "Kings Hwy" in station_name:
    #     station_tokens = ["Kings Highway"]
    # elif "Metropolitan Av / Lorimer St" in station_name:
    #     station_tokens = ["Metropolitan Av"]
    # elif station_tokens[0] == "Beach":
    #     station_tokens = [f"Beach {station_tokens[1]}th St"]
    # elif "Myrtle - Wyckoff Avs" in station_name:
    #     station_tokens = ["Myrtle Av"]
    # elif "Astoria Blvd" in station_name:
    #     station_tokens = ["Astoria Blvd-Hoyt Av"]
    # elif "Parkchester" in station_name:
    #     station_tokens = ["Parkchester-East 177th St"]
    # elif "111 St" in station_name:
    #     if set(station_lines) == set("A"):
    #         station_tokens = ["111th St-Greenwood Av"]
    #     else:
    #         station_tokens = ["111th St"]
    # elif "52 St" in station_name:
    #     station_tokens = ["Lincoln Av-52nd St"]
    # elif "14 St / 8 Av" in station_name:
    #     station_tokens = ["14th St"]
    # elif "Broadway Junction" in station_name:
    #     station_tokens = ["Broadway Junction-East New York"]
    # elif "80 St" in station_name:
    #     station_tokens = ["80th St-Hudson St"]
    # elif "88 St" in station_name:
    #     station_tokens = ["88th St-Boyd Av"]
    # elif "104 St" in station_name:
    #     if set(station_lines) == set("A"):
    #         station_tokens = ["104th St-Oxford Av"]
    #     if set(station_lines) == set("J Z"):
    #         station_tokens = ["104th St-102nd St"]
    # elif "Bay 50 St" in station_name:
    #     station_tokens = ["Bay 50th St"]
    # elif "Beverley Rd" in station_name:
    #     station_tokens = ["Beverly Rd"]
    # elif station_tokens[0] == "Avenue":
    #     station_tokens = ["Av", station_tokens[1]]

    #     if station_tokens == ["Av", "U"]:
    #         station_lines = "B Q"

    # elif "Briarwood" in station_name:
    #     station_tokens = ["Briarwood-Van Wyck Blvd"]
    # elif "Kosciuszko St" in station_name:
    #     station_tokens = ["Kosciusko St"]
    # elif "30 Av" in station_name:
    #     station_tokens = ["30 Av-Grand Av"]
    # elif "Queensboro Plaza" in station_name:
    #     station_lines += " Q"
    # elif "Dyckman St" in station_name:
    #     if set(station_lines) == set("A"):
    #         station_tokens = ["Dyckman St-200th St"]
    # elif "High St" in station_name:
    #     station_lines = "A C J Z 2 3 4 5"
    # elif "Franklin Av" in station_name:
    #     if set(station_lines) == set("A C S"):
    #         station_lines = "A C FS"
    # elif "Prospect Park" in station_name:
    #     station_lines = "B Q FS"
    # elif "Neck Rd" in station_name:
    #     station_lines = "B Q"
    # elif "Broad Channel" in station_name:
    #     station_lines = "A H"

    if "168 St - Washington Hts" in station_name:
        station_name = "168 St-Washington Hts"
    elif "59 St - Columbus Circle" in station_name:
        station_name = "59 St-Columbus Circle"
    elif "Times Sq - 42 St / Port Authority Bus Terminal" in station_name:
        station_name = "42 St-Port Authority Bus Terminal"
    elif "14 St / 6 Av" in station_name or "14 St / 8 Av" in station_name:
        station_name = "14 St"
    elif "South Ferry / Whitehall" in station_name:
        station_name = "Whitehall St-South Ferry"
    elif "149 St - Grand Concourse" in station_name:
        station_name = "149 St-Grand Concourse"
    elif "Chambers St / WTC / Park Place / Cortlandt St" in station_name:
        station_name = "Park Place"
    elif "Borough Hall / Court St" in station_name:
        station_name = "Borough Hall"
    elif "Franklin Av / Botanic Garden" in station_name:
        station_name = "Franklin Av"
    elif "161 St - Yankee Stadium" in station_name:
        station_name = "161 St-Yankee Stadium"
    elif "Lexington Av / 59 St" in station_name:
        station_name = "Lexington Av/59 St"
    elif "Lexington Av / 51 St" in station_name:
        station_name = "51 St"
    elif "Grand Central - 42 St" in station_name:
        station_name = "Grand Central-42 St"
    elif "14 St - Union Sq" in station_name:
        station_name = "14 St-Union Sq"
    elif "Broadway-Lafayette St / Bleecker St" in station_name:
        station_name = "Bleecker St"
    elif "Brooklyn Bridge-City Hall / Chambers St" in station_name:
        station_name = "Brooklyn Bridge-City Hall"
    elif "Jackson Hts-Roosevelt Av / 74 St" in station_name:
        station_name = "Jackson Hts-Roosevelt Av"
    elif "Court Sq - 23 St" in station_name:
        station_name = "Court Sq-23 St"
    elif "42 St - Bryant Pk / 5 Av" in station_name:
        station_name = "42 St-Bryant Pk"
    elif "W 4 St - Wash Sq" in station_name:
        station_name = "W 4 St-Wash Sq"
    elif "Jay St - MetroTech" in station_name:
        station_name = "Jay St-MetroTech"
    elif "62 St / New Utrecht Av" in station_name:
        station_name = "New Utrecht Av"
    elif "34 St - Herald Sq" in station_name:
        station_name = "34 St-Herald Sq"
    elif "Delancey St / Essex St" in station_name:
        station_name = "Delancey St-Essex St"
    elif "4 Av - 9 St" in station_name:
        station_name = "4 Av-9 St"
    elif "Metropolitan Av / Lorimer St" in station_name:
        station_name = "Metropolitan Av"
    elif "Myrtle - Wyckoff Avs" in station_name:
        station_name = "Myrtle-Wyckoff Avs"
    elif "Jamaica Center-Parsons / Archer" in station_name:
        station_name = "Jamaica Center-Parsons/Archer"
    elif "Lexington Av / 63 St" in station_name:
        station_name = "Lexington Av/63 St"
    elif "5 Av / 53 St" in station_name:
        station_name = "5 Av/53 St"
    elif "5 Av / 59 St" in station_name:
        station_name = "5 Av/59 St"
    
    if station_id == 17763:
        station_lines += " M"

    mta_stations = mta_all_stations[
        (mta_all_stations["Stop Name"] == station_name)
    ]

    found_station = False
    for _, candidate in mta_stations.iterrows():
        if set(candidate["Daytime Routes"]).issubset(set(station_lines)):
            station_latlongs.append(
                [
                    candidate["GTFS Latitude"],
                    candidate["GTFS Longitude"],
                    station_id,
                ]
            )
            found_station = True
            break

    if mta_stations.empty or not found_station:
        not_found_cnt += 1
        print(f"{station_id}, {station_name}, {station_lines}: not found")
        print("Candidates:")
        print(
            mta_all_stations[
                (mta_all_stations["Stop Name"] == f"{station_name}")
            ].to_markdown()
        )
        print()
print(not_found_cnt)

setup_sqlite.update_latlong(station_latlongs)

0


In [22]:
# rat_sv["total_voters"] = rat_sv[["so_many", "one_or_two", "none"]].sum(axis=1)
# rat_sv = rat_sv.sort_values("total_voters", ascending=False)

# rat_sv["% so_many"] = rat_sv["so_many"] / rat_sv["total_voters"]
# rat_sv["% one_or_two"] = rat_sv["one_or_two"] / rat_sv["total_voters"]
# rat_sv["% none"] = rat_sv["none"] / rat_sv["total_voters"]
# rat_sv["seen_rats"] = (3 * rat_sv["so_many"]) + (1.5 * rat_sv["one_or_two"])
# rat_sv["% rats"] = (rat_sv["so_many"] + rat_sv["one_or_two"]) / rat_sv["total_voters"]

# rat_sv.head(20)

sightings_df = pd.read_sql("SELECT * from sightings", setup_sqlite.get_connector())
stations_df = pd.read_sql("SELECT * from stations", setup_sqlite.get_connector())

sightings_df["date_start"] = pd.to_datetime(sightings_df["date_start"])
sightings_df["date_end"] = pd.to_datetime(sightings_df["date_end"])

In [23]:
print(stations_df.shape)
print(sightings_df.shape)
print(sightings_df.head(10))
print(sightings_df.info())

(429, 5)
(41242, 6)
   station_id  so_many  one_or_two  none date_start   date_end
0       17496        3           7     2 2023-10-06 2023-11-05
1       17700        5           8     3 2023-10-06 2023-11-05
2       17497        9          21    10 2023-10-06 2023-11-05
3       17872        8           6     5 2023-10-06 2023-11-05
4       20055       16          10    11 2023-10-06 2023-11-05
5       17921        8           8     7 2023-10-06 2023-11-05
6       17546       31          39    33 2023-10-06 2023-11-05
7       17699       19          21    19 2023-10-06 2023-11-05
8       17549       10          11    10 2023-10-06 2023-11-05
9       17725        8          14    11 2023-10-06 2023-11-05
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41242 entries, 0 to 41241
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   station_id  41242 non-null  int64         
 1   so_many     41242 non-null  in

In [24]:
def plot_station_trend(station_name, station_id=None):
    if not station_id:
        station_id = stations_df[stations_df["station_name"] == station_name][
            "station_id"
        ].values
        if len(station_id) > 1:
            print(
                "Station name conflict, this station requires a specific ID due to duplicate names"
            )
            return

    station_data = sightings_df[sightings_df["station_id"] == station_id[0]]
    melted_data = station_data.melt(
        ["station_id", "date_start", "date_end"],
        var_name="sighting_type",
        value_vars=["so_many", "one_or_two", "none"],
    )

    fig = px.line(
        melted_data, x="date_end", y="value", color="sighting_type", markers=True
    )
    fig.update_layout(
        title=f"30 day Trail of Rats Seen at {station_name} ({station_id[0]})",
        xaxis_tickformat="%d %B (%a)<br>%Y",
        yaxis_title="Sighting Count",
    )
    fig.show()

    print(pio.to_html(fig, include_plotlyjs=False))


plot_station_trend("231 St")

<html>
<head><meta charset="utf-8" /></head>
<body>
    <div>                            <div id="3b5600ee-94ed-4311-809d-105fce27fc4e" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("3b5600ee-94ed-4311-809d-105fce27fc4e")) {                    Plotly.newPlot(                        "3b5600ee-94ed-4311-809d-105fce27fc4e",                        [{"hovertemplate":"sighting_type=so_many\u003cbr\u003edate_end=%{x}\u003cbr\u003evalue=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"so_many","line":{"color":"#636efa","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines+markers","name":"so_many","orientation":"v","showlegend":true,"x":["2023-11-05T00:00:00","2023-11-06T00:00:00","2023-11-07T00:00:00","2023-11-09T00:00:00","2023-11-10T00:00:00","2023-11-11T00:00:00","2023-11-12T

In [32]:
def plot_per_day_delta(station_name, station_id=None):
    if not station_id:
        station_id = stations_df[stations_df["station_name"] == station_name][
            "station_id"
        ].values
        if len(station_id) > 1:
            print(
                "Station name conflict, this station requires a specific ID due to duplicate names"
            )
            return

    station_data = sightings_df[sightings_df["station_id"] == station_id[0]]
    station_data["total_sightings"] = (
        station_data["so_many"] + station_data["one_or_two"]
    )
    station_data["total_records"] = (
        station_data["so_many"] + station_data["one_or_two"] + station_data["none"]
    )
    station_data["perc_sightings"] = (
        station_data["total_sightings"] / station_data["total_records"]
    )
    station_data = station_data.drop(
        columns=["station_id", "so_many", "one_or_two", "none"]
    )

    station_data["date_delta"] = station_data["date_end"] - station_data["date_end"].shift(1)
    station_data["sightings_delta"] = station_data["total_sightings"] - station_data["total_sightings"].shift(1)
    # station_data["date_delta"] = pd.to_numeric(station_data["date_delta"].dt.days, downcast="integer")
    station_data["date_delta"] = station_data["date_delta"].dt.days

    # melted_data = station_data.melt(
    #     ["station_id", "date_start", "date_end"],
    #     var_name="sighting_type",
    #     value_vars=["so_many", "one_or_two", "none"],
    # )

    # print(melted_data)

    print(station_data)

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(
        go.Scatter(
            x=station_data["date_end"],
            y=station_data["total_sightings"],
            name="Total Sightings",
            mode="lines+markers",
            text=[
                f"date_end: {date.strftime('%d %B (%a)')}"
                for date in station_data["date_end"]
            ],
        ),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(
            x=station_data["date_end"],
            y=station_data["total_records"],
            name="Total Records",
            mode="lines+markers",
            text=[
                f"date_end: {date.strftime('%d %B (%a)')}"
                for date in station_data["date_end"]
            ],
        ),
        secondary_y=False,
    )

    fig.add_trace(
        go.Bar(
            x=station_data["date_end"],
            y=station_data["perc_sightings"],
            name="Percent sightings of total records",
            opacity=0.25,
        ),
        secondary_y=True,
    )

    # fig.add_trace(
    #     go.Scatter(
    #         x=station_data["date_end"],
    #         y=station_data["date_delta"]
    #     )
    # )

    # fig.add_trace(
    #     go.Scatter(
    #         x=station_data["date_end"],
    #         y=station_data["sightings_delta"]
    #     )
    # )

    # fig.update_xaxes(title_text="Date End")
    # fig.update_yaxes(title_text="Count")

    fig.update_layout(
        title_text=f"30 Day Trail of Rat Sightings - {station_name}",
        yaxis=dict(
            title=dict(text="Count"),
            side="left",
        ),
        yaxis2=dict(
            title=dict(text="%"),
            side="right",
            range=[0, 1],
            overlaying="y",
            tickmode="sync",
        ),
    )

    return fig


plot_per_day_delta("231 St")


      date_start   date_end  total_sightings  total_records  perc_sightings  date_delta  sightings_delta
340   2023-10-06 2023-11-05                3             20        0.150000         NaN              NaN
724   2023-10-07 2023-11-06                3             22        0.136364         1.0              0.0
1112  2023-10-08 2023-11-07                3             22        0.136364         1.0              0.0
1504  2023-10-10 2023-11-09                3             23        0.130435         2.0              0.0
1887  2023-10-11 2023-11-10                3             21        0.142857         1.0              0.0
...          ...        ...              ...            ...             ...         ...              ...
39608 2024-01-29 2024-02-28                6             15        0.400000         1.0              0.0
39919 2024-01-30 2024-02-29                6             13        0.461538         1.0              0.0
40323 2024-01-31 2024-03-01                5           

In [26]:
def plot_summary():
    nyc_total_sightings = (
        sightings_df.groupby(["date_start", "date_end"])
        .sum(["so_many", "one_or_two", "none"])
        .drop(columns="station_id")
        .reset_index()
    )
    melted_data = nyc_total_sightings.melt(
        ["date_start", "date_end"],
        var_name="sighting_type",
        value_vars=["so_many", "one_or_two", "none"],
    )

    fig = px.line(
        melted_data, x="date_end", y="value", color="sighting_type", markers=True
    )
    fig.update_layout(
        title="30 day Trail of Total Rats Seen",
        xaxis_tickformat="%d %B (%a)<br>%Y",
        yaxis_title="Sighting Count",
    )
    fig.show()


plot_summary()